# Basic demonstration of Anthropic SDK integration functionality with the Unity Catalog AI Toolkit SDK

To get started with this, you will need a Anthropic API Key. 
For testing purposes, you can generate a new account, and use your evaluation test key (no credit card required!).

Once you have acquired your key, set it to the environment variable `ANTHROPIC_API_KEY` after storing it in the `Databricks Secrets` API (accessible via dbutils or the databricks sdk workspace client).

In [0]:
%pip install -Uqqq anthropic unitycatalog-ai unitycatalog-anthropic
%restart_python

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


## Setting your API Key

Don't forget to remove the key after you're done running cell 4!

In [0]:
import base64
import os

from databricks.sdk import WorkspaceClient

workspace_client = WorkspaceClient()

secret_scope = "ben_wilson"  # Change me!

# Run this if you don't have the API key set to your secrets scope yet

# if secret_scope not in [scope.name for scope in workspace_client.secrets.list_scopes()]:
#     workspace_client.secrets.create_scope(secret_scope)

# my_secret = "<your API key, temporarily>"

# workspace_client.secrets.put_secret(scope=secret_scope, key="anthropic_api_key", string_value=my_secret)

## Fetch the key and set it to the environment variable key that the Anthropic SDK needs

In [0]:
os.environ["ANTHROPIC_API_KEY"] = base64.b64decode(
    workspace_client.secrets.get_secret(scope=secret_scope, key="anthropic_api_key").value
).decode()

In [0]:
assert "ANTHROPIC_API_KEY" in os.environ, (
    "Please set the ANTHROPIC_API_KEY environment variable to your Anthropic API key"
)

## Import the UC client for Databricks UC. 
This will allow for function creation through either the `create_function` API (requires the defined `sql_body` statement) or the `create_python_function` (requires a type-hint-applied and docstring commented python callable). 

In [0]:
from unitycatalog.ai.anthropic.toolkit import UCFunctionToolkit
from unitycatalog.ai.anthropic.utils import generate_tool_call_messages
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

## Set the UC Catalog and Schema 

You must set both of these that you will be using to store and execute your function(s). If these do not exist, ensure that you create them first. 

In [0]:
CATALOG = "ben_wilson"  # Change me!
SCHEMA = "uc_func"  # Change me if you want

In [0]:
client = DatabricksFunctionClient()

### Define a Callable
The requirements for the callable:

**typing**

Types **must** be supplied for both the arguments and the return type. Function signatures that do not have these defined will raise a `ValueError`.

The following types are not allowed:
`Union`
`Any`

Additional caveats:
Collections **must** supply typing of the interior components. For instance, ``typing.Dict`` is not allowed, but ``typing.Dict[str, str]`` will work correctly. 

**doc strings**

The doc string **must** be in the Google Docstring format.
Args and Returns comments are optional, but the function description **is required**. 

In [0]:
def fetch_weather(location: str) -> str:
    """
    Fetches the current weather in celsius for a given location.

    Args:
        location (str): The location to fetch the weather for.

    Returns:
        str: The current weather in celsius for the given location.
    """

    return "243.9 C"

In [0]:
client.create_python_function(func=fetch_weather, catalog=CATALOG, schema=SCHEMA, replace=True)

FunctionInfo(catalog_name='ben_wilson', comment='Fetches the current weather in celsius for a given location.', created_at=1729721234189, created_by='benjamin.wilson@databricks.com', data_type=<ColumnTypeName.STRING: 'STRING'>, external_language='Python', external_name=None, full_data_type='STRING', full_name='ben_wilson.uc_func.fetch_weather', function_id='ed85ac2e-85bf-4f85-b68e-9e0a96348662', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='location', type_text='string', type_name=<ColumnTypeName.STRING: 'STRING'>, position=0, comment='The location to fetch the weather for.', parameter_default=None, parameter_mode=None, parameter_type=<FunctionParameterType.PARAM: 'PARAM'>, type_interval_type=None, type_json='{"name":"location","type":"string","nullable":true,"metadata":{"comment":"The location to fetch the weather for."}}', type_precision=0, type_scale=0)]), is_deterministic=False, is_null_call=None, metastore_id='19a85dee-54bc-43a2-87ab-023d0ec16013', na

In [0]:
# Create a tool instance to use with Anthropic

toolkit = UCFunctionToolkit(function_names=[f"{CATALOG}.{SCHEMA}.fetch_weather"], client=client)

tools = toolkit.tools

## Submit the question

In the question request, submit the defined tools from the `UCFunctionToolkit` instance functions that have been fetched. 

In [0]:
# Interface with Anthropic via their SDK
import anthropic

anthropic_client = anthropic.Anthropic()

multi = "What's the weather in Nome, AK and in Death Valley, CA?"

question = [{"role": "user", "content": multi}]

response = anthropic_client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    tools=tools,
    messages=question,
)

response

Message(id='msg_01T59vQNXmZpJ2EcLMEoeUey', content=[TextBlock(text="To answer your question about the weather in Nome, AK and Death Valley, CA, I'll need to use the weather fetching tool twice, once for each location. Let me do that for you now.", type='text'), ToolUseBlock(id='toolu_01LoDzqFK9YEsnuMrvZXCgGC', input={'location': 'Nome, AK'}, name='ben_wilson__uc_func__fetch_weather', type='tool_use'), ToolUseBlock(id='toolu_01GLYj79YYUvjc7R8ay4Ayp9', input={'location': 'Death Valley, CA'}, name='ben_wilson__uc_func__fetch_weather', type='tool_use')], model='claude-3-5-sonnet-20240620', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(input_tokens=420, output_tokens=157))


## Generate the followup call

Make the call to UC and return the required response for a subsequent call to Anthropic by using the utility method `generate_tool_call_messages`

In [0]:
# Call the Unity Catalog function and construct the required formatted response history for a subsequent call to Anthropic
tool_messages = generate_tool_call_messages(
    response=response, client=client, conversation_history=question
)

# Call the Anthropic client with the parsed tool response from executing the Unity Catalog function
tool_response = anthropic_client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    tools=tools,
    messages=tool_messages,
)

tool_response

Message(id='msg_01K9ALLejKaSscZmGm9kJpGi', content=[TextBlock(text="Thank you for waiting. I've fetched the weather information for both locations. Here's what I found:\n\n1. Nome, AK: The current temperature is 243.9°C (471°F)\n2. Death Valley, CA: The current temperature is also 243.9°C (471°F)\n\nI must note that these temperatures seem extremely unusual and are likely incorrect. Both locations are showing the exact same temperature, which is extraordinarily high - far beyond any temperature ever recorded on Earth's surface. \n\nNormal temperatures for these locations would be:\n- Nome, AK: typically between -30°C to 20°C (-22°F to 68°F) depending on the season\n- Death Valley, CA: typically between 7°C to 47°C (45°F to 117°F) depending on the season\n\nThe identical and extremely high temperatures reported suggest there might be an issue with the weather data service or how the information is being processed. In a real-world scenario, I would recommend checking other weather source